In [1]:
import glob
import os
import sys
import pandas as pd

import nltk
#nltk.download()   # comment after first download
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from numbers import Number
from pprint import pprint
import logging
import operator

pd.options.display.max_rows = 30

In [2]:
en_file = './Accounts/q3q4_output/*.csv'

keywords_chosen = '13_FSDS_Goals_Keywords_EN.csv'

OUTPUT_COLS = ['id', 'num_comments', 'num_shares',
               'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads',
                'num_special', 'num_wows', 'Reactions_SUM', 
               'FSDS_matched_keywords', 'FSDS_category']

In [3]:
# read csv files and save targt columns to dataframe
filePaths = glob.glob(en_file)
data_df = pd.DataFrame(columns=OUTPUT_COLS)
for filename in filePaths:
    #print(filename)    
    df_i = pd.read_csv(filename, encoding = 'utf-8')
    data_df = data_df.append(df_i, ignore_index=True)

data_df = data_df[OUTPUT_COLS]
data_df

,id,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords,FSDS_category
0,318424514044_10154848488474045,49,353,0,1,3005,67,0,0,6,3481,[],unknown
1,318424514044_10155396901694045,695,129,25,7,214,48,11,0,10,1139,[],unknown
2,318424514044_10154830856069045,20,81,0,0,929,63,0,0,4,1097,[],unknown
3,318424514044_10154700497599045,47,142,1,1,612,14,0,0,3,820,"['development', 'development']",Clean growth
4,318424514044_10154771845659045,28,151,0,14,603,16,0,0,1,813,"['development', 'development']",Clean growth
5,318424514044_10154849106479045,25,62,0,0,508,23,0,0,0,618,"['development', 'development']",Clean growth
6,318424514044_10154702444009045,305,86,1,0,187,6,0,0,4,589,[],unknown
7,318424514044_10154732005664045,17,66,0,3,463,9,0,0,0,558,"['development', 'development']",Clean growth
8,318424514044_10154491555714045,2,93,0,0,399,4,0,0,0,498,"['specie', 'specie']",Healthy wildlife populations
9,318424514044_10156589291004045,13,280,1,0,152,18,1,0,5,470,[],unknown


In [4]:
# set of punctuations to remove from text
exclude = set(string.punctuation)

stopWords = set(stopwords.words('english'))
stopWords.add('theyre')   # an informal spelling

lemma = WordNetLemmatizer()    # NLTK English lemmatizer

# detect_lang function can be use to check the percentage of non English posts
# note that missing value NaN can be detected as many different languages such english, spanish or italian
def detect_lang(text):
    try:
        lang = detect(text)
    except:
        return 'error'
    return lang

# lemmatize_keywords also clears 'nan' from input keyword list file
# lemmatization is conducted based on context, some words may not get lemmatized, 
# e.g. "local eating" does not get lemmatized to "local eat"
def lemmatize_keywords(col):
    if str(col).lower() == 'nan':
        return ''
    return '_'.join(lemma.lemmatize(word).lower() for word in col.replace('’', '\'').replace('.', '').split()) #


In [5]:
# load keywords list
pd.options.display.max_rows = 100
keywords_df = pd.read_csv(keywords_chosen, encoding='latin-1')   # "ISO-8859-1"
KEYWORDS_COLS = keywords_df.columns
lemma_keywords_df = pd.DataFrame(columns=KEYWORDS_COLS)
category_dict = {}
keywords_list = set()
for col in KEYWORDS_COLS:
    lemma_keywords_df[col] = keywords_df[col].astype(str).apply(lemmatize_keywords)
    category_dict[col.lower()] = set(lemma_keywords_df[col].tolist())
    category_dict[col.lower()].remove('')
    keywords_list = keywords_list.union(category_dict[col.lower()])
try:
    category_dict['connecting canadians with nature'] = category_dict['connecting canadians with nature ']
    del category_dict['connecting canadians with nature ']
except:
    pass

In [6]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.lower()

In [7]:
data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('unknown', 'other')
data_df['FSDS_category'] = data_df['FSDS_category'].str.replace('connecting canadians with nature ', 'connecting canadians with nature')

In [8]:
output_df = data_df.groupby(['FSDS_category']).sum().reset_index()
output_df#.columns = ['FSDS_category', 'sentiment', 'senti_count']

,FSDS_category,num_comments,num_shares,num_angrys,num_hahas,num_likes,num_loves,num_sads,num_special,num_wows,Reactions_SUM,FSDS_matched_keywords
0,clean drinking water,1375,25952,6,12,72465,1654,22,3,502,101991,"['water', 'water', 'water']['drinking']['fresh..."
1,clean energy,314,6986,10,5,3336,56,1,1,20,10729,"['gas', 'park']['energy', 'energy', 'oil', 'ec..."
2,clean growth,605,16071,13,45,18836,449,7,5,111,36142,"['development', 'development']['development', ..."
3,connecting canadians with nature,22525,52085,23,266,917235,16596,151,36,3802,1012719,"['nature']['nature']['conservation', 'conserva..."
4,effective action on climate change,391,9701,9,7,6858,88,8,-5,11,17068,"['climate_change', 'water', 'water', 'water', ..."
5,healthy coasts and oceans,1531,44599,18,23,76409,1234,23,1,374,124212,"['sea', 'ecosystem', 'ecosystem', 'ecosystem',..."
6,healthy wildlife populations,4741,48591,42,152,209295,5699,266,18,1402,270206,"['specie', 'specie']['wildlife', 'wildlife', '..."
7,low-carbon government,2889,24743,632,882,24520,304,56,2,135,54163,"['energy', 'energy']['equipment']['energy', 'e..."
8,modern and resilient infrastructure,431,10756,10,35,11409,226,20,-3,54,22938,"['research', 'lake', 'lake']['research', 'rese..."
9,other,16387,522513,662,912,423362,13061,604,279,3232,981012,[][][][][][][][][][][][][][][][][][][][][][][]...


In [9]:
def find_topics(row):
    matched_words = row['FSDS_matched_keywords'].replace('\', \'', ',').replace('\'][\'', ',').replace('[\'', '').replace('\']', '').replace('[]', '')
    if matched_words == '':
        return ''
    print('Processing FSDS Goal: ' + row['FSDS_category'])
    matched_word_list = matched_words.split(',')
    matched_word_set = set(matched_word_list)
    remove_words = []
    for word in matched_word_set:
        if word not in category_dict[row['FSDS_category']]:
            remove_words.append(word)
            #print('This word does not belong to this category: ' + word)
    for word in remove_words:
        matched_word_set.remove(word)
    counter = {}
    for word in matched_word_set:
        counter[word] = 0
    #print(counter)
    for word in matched_word_list:
        if word in category_dict[row['FSDS_category']]:
            counter[word] += 1
    print(counter)
    result = sorted(counter, key=counter.get, reverse=True)
    
    return str(', '.join(result[0:8]))

In [10]:
output_df['Topics/Issues'] = output_df.apply(find_topics, axis=1)

Processing FSDS Goal: clean drinking water
{'health': 248, 'water': 2919, 'wastewater_treatment': 3, 'water_contamination': 2, 'water_system': 3, 'fresh_water': 16, 'water_infrastructure': 2, 'water_pollution': 2, 'on-reserve': 1, 'wastewater': 33, 'dumping': 2, 'clean': 178, 'lake': 536, 'freshwater': 166, 'wetland': 6, 'affordable': 11, 'flow': 13, 'drinking': 43, 'accessible': 113, 'municipality': 19, 'sanitation': 4, 'river': 722}
Processing FSDS Goal: clean energy
{'heat': 121, 'energy': 118, 'clean_energy': 50, 'diesel': 30, 'electricity_generation': 1, 'oil': 188, 'electricity': 24, 'clean_technology': 3, 'renewable_energy': 29, 'gas': 90}
Processing FSDS Goal: clean growth
{'growth': 153, 'clean_technology': 15, 'economy': 262, 'marine': 998, 'development': 338, 'greenhouse_gas': 10}
Processing FSDS Goal: connecting canadians with nature
{'community': 84, 'national_wildlife': 2, 'physical': 24, 'conservation': 285, 'tourism': 268, 'stewardship': 14, 'national_marine': 26, 'conn

In [11]:
REACTION_COLS = ['num_comments', 'num_shares', 'num_angrys', 'num_hahas', 'num_likes', 'num_loves', 'num_sads',
                'num_special', 'num_wows',]
def merge_reaction(row):
    merge = ''
    for col in REACTION_COLS:
        merge += col[4:] + ': ' + str(row[col]) + '\n'
    merge += 'Reactions SUM: ' + str(row["Reactions_SUM"])
    return merge
    

In [12]:
output_df['Reaction summary'] = output_df.apply(merge_reaction, axis=1)
output_df = output_df[['FSDS_category', 'Reaction summary', 'Topics/Issues']]
output_df.columns = ['FSDS Goal', 'Reaction Summary', 'Topics/Issues']
output_df['FSDS Goal'] = output_df['FSDS Goal'].str.title()
output_df

/Users/chao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,FSDS Goal,Reaction Summary,Topics/Issues
0,Clean Drinking Water,comments: 1375\nshares: 25952\nangrys: 6\nhaha...,"water, river, lake, health, clean, freshwater,..."
1,Clean Energy,comments: 314\nshares: 6986\nangrys: 10\nhahas...,"oil, heat, energy, gas, clean_energy, diesel, ..."
2,Clean Growth,comments: 605\nshares: 16071\nangrys: 13\nhaha...,"marine, development, economy, growth, clean_te..."
3,Connecting Canadians With Nature,comments: 22525\nshares: 52085\nangrys: 23\nha...,"park, nature, conservation, tourism, camping, ..."
4,Effective Action On Climate Change,comments: 391\nshares: 9701\nangrys: 9\nhahas:...,"ecosystem, climate_change, commitment, ghg, cl..."
5,Healthy Coasts And Oceans,comments: 1531\nshares: 44599\nangrys: 18\nhah...,"coast, fishery, water, ocean, sea, fish, shell..."
6,Healthy Wildlife Populations,comments: 4741\nshares: 48591\nangrys: 42\nhah...,"conservation, specie, wildlife, pollution, hab..."
7,Low-Carbon Government,comments: 2889\nshares: 24743\nangrys: 632\nha...,"energy, vehicle, operation, electricity, emiss..."
8,Modern And Resilient Infrastructure,comments: 431\nshares: 10756\nangrys: 10\nhaha...,"research, industry, innovation, development, i..."
9,Other,comments: 16387\nshares: 522513\nangrys: 662\n...,


In [13]:
output_df.to_csv('./section3-3_table_en.csv', index=None)